In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import re
import numpy as np 
import pandas as pd

In [17]:
Data_Dir = '../input/quora-question-pairs/'
#Word_Vec_Dir = '../input/glove6b300dtxt/'
Word_Vec_Dir = '../input/glove6b50dtxt/'
Embedding_File = Word_Vec_Dir + 'glove.6B.50d.txt'
Train_Data_File = Data_Dir + 'train.csv.zip'
Test_Data_File = Data_Dir + 'test.csv.zip'

In [4]:
df_train = pd.read_csv(Train_Data_File, encoding='utf-8')
print(df_train.shape)
df_train.sample(5)

In [5]:
df_train = df_train.drop_duplicates(ignore_index=True).sample(5000)

In [6]:
df = pd.DataFrame(df_train['question1'].append(df_train['question2']),columns=['questions']).drop_duplicates(ignore_index=True)
print(df.shape)
df.head()

In [7]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer('english')

def clean_text(text, remove_stopwords = True, stem_words = True):
    
    text = str(text).lower().split()

    if remove_stopwords:
        text = [w for w in text if not w in stop_words]
    
    text = " ".join(text)


    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", "", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    
    
    if stem_words:
        text = text.split()
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    return(text)

In [8]:
df['cleaned_ques'] = df['questions'].apply(lambda x :clean_text(x,remove_stopwords = False, stem_words=False))
df.sample(10)

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
df['#words cleaned'] = df["cleaned_ques"].apply(lambda x: len(str(x).split()))
sns.distplot(df['#words cleaned'])
plt.show()
sns.boxplot(df['#words cleaned'])
plt.show()

In [12]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
#from keras.utils import plot_model
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, LSTM, Lambda
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalAveragePooling1D
import keras.backend as K

In [13]:
Max_Sequence_Length = 25
Embedding_Dim = 50

tokenizer = Tokenizer()#num_words=Max_Num_Words)
tokenizer.fit_on_texts(df['cleaned_ques'])

seq = tokenizer.texts_to_sequences(df['cleaned_ques'])

In [14]:
word_index = tokenizer.word_index
print('{} unique tokens are found'.format(len(word_index)))

In [15]:
train_data = pad_sequences(seq, maxlen=Max_Sequence_Length,padding='pre')
train_data

In [18]:
print('Create word embedding dictionary')
embeddings_index = {}
f = open(Embedding_File, encoding='utf-8')
# for line in tqdm(f):
for line in f:
    values = line.split()
    # word = values[0]
    word = ''.join(values[:-50])   
    # coefs = np.asarray(values[1:], dtype='float32')
    coefs = np.asarray(values[-50:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found {} word vectors of glove.'.format(len(embeddings_index)))

In [19]:
#Max_Num_Words = 85345
#num_words = min(Max_Num_Words, len(word_index))+1
num_words = len(word_index)+1
hits=0
miss=0
embedding_matrix = np.zeros((num_words, Embedding_Dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits=hits+1
    else:
        miss=miss+1
                
#print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))
print('hits:',hits)
print("misses:",miss)

In [20]:
embedding_matrix.shape

In [21]:

temp = np.zeros((train_data.shape[0], train_data.shape[1], 50))

for i in range(train_data.shape[0]):
    if i % 10000==0:
        print(i)
    for j in range(train_data.shape[1]):
        if train_data[i][j]!=0:
            temp[i][j] = embedding_matrix[train_data[i][j]]

temp.shape

In [39]:
'''
model = Sequential()
model.add(LSTM(256, activation='relu', input_shape=(40,300)))
model.add(RepeatVector(40))
model.add(LSTM(256, activation='relu', return_sequences=True))
#model.add(TimeDistributed(Dense(300)))
model.compile(optimizer='adam', loss='mse')
'''


In [40]:
#hist = model.fit(temp, temp, epochs=50, verbose=1,batch_size = 2)

In [24]:
embedded_inputs = Input(shape=(25, 50))
encoded_inputs = LSTM(30, name="encoder")(embedded_inputs)

decoded = RepeatVector(25)(encoded_inputs)

decoded = LSTM(30, return_sequences=True)(decoded) 

output =  TimeDistributed(Dense(50))(decoded)
autoencoder = Model(embedded_inputs, output)
autoencoder.compile(loss='mse', optimizer='adam') # we use 'mse' loss function to train our autoencoder

print(autoencoder.summary())



In [25]:
checkpoint = ModelCheckpoint(filepath= "models/autoencoder.h5", save_best_only=True)
autoencoder.fit(temp, temp,batch_size=5, epochs=100, verbose=1, validation_split=0.2, callbacks=[checkpoint])

In [27]:
context_vector = Model(autoencoder.input, autoencoder.get_layer("encoder").output)
vec = context_vector.predict(temp)

In [33]:
len(vec)

In [37]:
print(vec[8003])
df['questions'][8003]

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
sim={}
for i in range(0,len(vec)):
    sim[i] = cosine_similarity([vec[8003]],[vec[i]]) 

In [53]:
top_sim = sorted(sim, key=sim.get, reverse=True)[:10]
for i in top_sim:
    print(df['questions'][i])

In [54]:
sns.distplot(list(sim.values()))